##Problem Description: 
end2end well annotated and commented demo of continual learning using avalanche library

SUBMITTED BY: SWATHI ANANDRAM



Avalanche is an End-to-End Continual Learning Library (now part of the PyTorch Ecosystem!) powered by ContinualAI with the unique goal of providing a shared and collaborative open-source (MIT licensed) codebase for fast prototyping, training and reproducible evaluation of continual learning algorithms.

Avalanche can help Continual Learning researchers and practitioners in several ways:

* Write less code, prototype faster & reduce errors
* Improve reproducibility
* Improve modularity and reusability
* Increase code efficiency, scalability & portability
* Augment impact and usability of their research products

The following demo shows how the code changes by using Avalanche

###Installing Avalanche

In [ ]:
!pip install avalanche-lib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 582 kB 5.6 MB/s 
     |████████████████████████████████| 1.9 MB 15.8 MB/s 
     |████████████████████████████████| 427 kB 54.6 MB/s 
     |████████████████████████████████| 529 kB 44.9 MB/s 
     |████████████████████████████████| 532 kB 29.0 MB/s 
     |████████████████████████████████| 182 kB 55.7 MB/s 
     |████████████████████████████████| 166 kB 42.4 MB/s 
     |████████████████████████████████| 63 kB 908 kB/s 
     |████████████████████████████████| 166 kB 51.5 MB/s 
     |████████████████████████████████| 162 kB 55.4 MB/s 
     |████████████████████████████████| 162 kB 52.8 MB/s 
     |████████████████████████████████| 158 kB 58.3 MB/s 
     |████████████████████████████████| 157 kB 23.5 MB/s 
     |████████████████████████████████| 157 kB 54.1 MB/s 
     |████████████████████████████████| 157 kB 54.3 MB/s 
     |████████████████████████████

In [ ]:
import avalanche
avalanche.__version__

'0.2.1'

###Without using Avalanche

Let's see here what it takes to run a Permuted MNIST experiment without Avalanche:

In [ ]:
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, RandomCrop
from torch.utils.data import DataLoader
import numpy as np
from copy import copy

# Config
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# model
class SimpleMLP(nn.Module):

    def __init__(self, num_classes=10, input_size=28*28):
        super(SimpleMLP, self).__init__()

        self.features = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
        )
        self.classifier = nn.Linear(512, num_classes)
        self._input_size = input_size

    def forward(self, x):
        x = x.contiguous()
        x = x.view(x.size(0), self._input_size)
        x = self.features(x)
        x = self.classifier(x)
        return x
model = SimpleMLP(num_classes=10)

# CL Benchmark Creation
list_train_dataset = []
list_test_dataset = []
rng_permute = np.random.RandomState(0)
train_transform = transforms.Compose([
    RandomCrop(28, padding=4),
    ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
test_transform = transforms.Compose([
    ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# permutation transformation
class PixelsPermutation(object):
    def __init__(self, index_permutation):
        self.permutation = index_permutation

    def __call__(self, x):
        return x.view(-1)[self.permutation].view(1, 28, 28)

def get_permutation():
    return torch.from_numpy(rng_permute.permutation(784)).type(torch.int64)

# for every incremental step
permutations = []
for i in range(3):
    # choose a random permutation of the pixels in the image
    idx_permute = get_permutation()
    current_perm = PixelsPermutation(idx_permute)
    permutations.append(idx_permute)

    # add the permutation to the default dataset transformation
    train_transform_list = train_transform.transforms.copy()
    train_transform_list.append(current_perm)
    new_train_transform = transforms.Compose(train_transform_list)

    test_transform_list = test_transform.transforms.copy()
    test_transform_list.append(current_perm)
    new_test_transform = transforms.Compose(test_transform_list)

    # get the datasets with the constructed transformation
    permuted_train = MNIST(root='./data/mnist',
                           download=True, transform=new_train_transform)
    permuted_test = MNIST(root='./data/mnist',
                    train=False,
                    download=True, transform=new_test_transform)
    list_train_dataset.append(permuted_train)
    list_test_dataset.append(permuted_test)

# Train
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

for task_id, train_dataset in enumerate(list_train_dataset):

    train_data_loader = DataLoader(
        train_dataset, num_workers=4, batch_size=32)
    
    for ep in range(2):
        for iteration, (train_mb_x, train_mb_y) in enumerate(train_data_loader):
            optimizer.zero_grad()
            train_mb_x = train_mb_x.to(device)
            train_mb_y = train_mb_y.to(device)

            # Forward
            logits = model(train_mb_x)
            # Loss
            loss = criterion(logits, train_mb_y)
            # Backward
            loss.backward()
            # Update
            optimizer.step()

# Test
acc_results = []
for task_id, test_dataset in enumerate(list_test_dataset):
    
    test_data_loader = DataLoader(
        test_dataset, num_workers=4, batch_size=32)
    
    correct = 0
    for iteration, (test_mb_x, test_mb_y) in enumerate(test_data_loader):

        # Move mini-batch data to device
        test_mb_x = test_mb_x.to(device)
        test_mb_y = test_mb_y.to(device)

        # Forward
        test_logits = model(test_mb_x)

        # Loss
        test_loss = criterion(test_logits, test_mb_y)

        # compute acc
        correct += test_mb_y.eq(test_logits.argmax(dim=1)).sum().item()
    
    acc_results.append(correct / len(test_dataset))

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


###With Avalanche

In [ ]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive

scenario = SplitMNIST(n_experiences=5)

# MODEL CREATION
model = SimpleMLP(num_classes=scenario.n_classes)

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [2, 3]
-- >> Start of training phase << --


/usr/local/lib/python3.7/dist-packages/avalanche/training/plugins/evaluation.py:82: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  "No benchmark provided to the evaluation plugin. "


100%|██████████| 25/25 [00:05<00:00,  4.48it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 602867.0586
	DiskUsage_MB/train_phase/train_stream/Task000 = 602867.0586
	Loss_Epoch/train_phase/train_stream/Task000 = 1.2091
	Loss_MB/train_phase/train_stream/Task000 = 0.3608
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0073
	Time_Epoch/train_phase/train_stream/Task000 = 5.5141
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6657
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9101
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 21/21 [00:02<00:00,  9.23it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 97.1824
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 602871.6455
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.3115
	